In [ ]:
## python modules used within this notebook
import numpy as np
from scipy import integrate
from scipy import interpolate
import matplotlib.pyplot as plt
import matplotlib.animation
import matplotlib.colors as colors
import os
import h5py
import sys
import MMA_administration as MMA
import mynumerics as mn
import units
from IPython.display import display, Markdown
from IPython.display import HTML

import dataformat_CUPRAD as dfC
import HHG
import plot_presets as pp 

matplotlib.rcParams['animation.embed_limit'] = 200.
# print(matplotlib.rcParams['animation.embed_limit'])


# %%capture
# %matplotlib inline
# import mpld3
# mpld3.enable_notebook()

# %matplotlib agg
%matplotlib inline

## TDSE with a custom input

Now, we move to the microscopic response. We show the interface for the TDSE solver accessed directly through Python. We use this solver for a custom field we define, and then analyse the result in details. We will show the spectrum of the source term, wavefunction, we do energetic analyses via the Gabor transform and [invariant energetic distribution](https://doi.org/10.1103/PhysRevA.106.053115). FInally, we will show also the depletion of the ground state.


First, we import the compiled dynamical library and its Pythonic wrapper:

In [ ]:
## ! Use environ variables here


# Main Python TDSE helper script, contains C structures, function wrappers etc.
# sys.path.append('/mnt/d/git/CUPRAD_TDSE_Hankel/1DTDSE') # shall be already in environment now
from PythonTDSE import *

# Compiled dynamic C library
path_to_DLL = os.path.join(os.environ['TDSE_1D_BUILD'],'libsingleTDSE.so') # '/mnt/d/git/CUPRAD_TDSE_Hankel/1DTDSE/build/libsingleTDSE.so'
DLL = TDSE_DLL(path_to_DLL)

### Define the custom input field & numerical parameters

Here we define the input parameters for the CTDSE solver and the initial pulse. We show an example of a chirped pulse with a $\sin^2$-envelope. The field is then given by

$$ \mathcal{E}(t) = \mathcal{E}_0 \sin^2 \left( \frac{t}{T_{\text{envelope}}} \right) \cos \left(\omega_0 t + \omega_c t^2 \right) \,.$$

(Note that the instantaneous frequency is then $\omega_i(t) = \omega_0 + 2\omega_c t$. This means that $\omega_0$ cannot be generally taken as the central frequency.)

In [ ]:
omega0 = mn.ConvertPhoton(1000e-9,'lambdaSI','omegaau')
chirp = 2e-4
E_0 = 0.15   # peak electric field amplitude

T0 = mn.ConvertPhoton(omega0,'omegaau','T0au') # the duration of the reference cycle
T_max = 3*T0 # total pulse duration expressed in the number of the reference cycles
N_t = 10000  # # of points for field construction (not for TDSE)

# Construct the field
tgrid = np.linspace(0, T_max, N_t)
E = E_0* (np.sin(np.pi*tgrid/T_max)**2) *np.cos(omega0*tgrid + chirp*(tgrid)**2)


# Create instance of input structure
inputs = inputs_def()

# Set the inputs for the TDSE solver
trg_a = 1.1893 # Argon 
inputs.init_default_inputs(
            Eguess   = -0.5145 ,
            trg_a    = trg_a ,     
            dt       = 0.125 ,
            dx       = 0.4 ,
            num_r    = 16000 ,
            writewft = 1 ,
            tprint   = 1. ,
            x_int    = 2. )
# Note: Parameters currently needs to be fixed for the gauge-invariant energetic analysis (gas & some of numerics for the same ensemble of bound states)

### Pipeline to execute the TDSE computation

In [ ]:
inputs.init_time_and_field(DLL, E = E, t = tgrid) # set our electric field as the input
DLL.init_GS(inputs)                               # create the C-types input for the C-library
output = outputs_def()                            # prepare the structure that holds the TDSE outputs 
DLL.call1DTDSE(inputs, output)                    # run TDSE

### Obtain detailed analyses and visualisation
Here we specify some parameters for various analyses and plotting

In [ ]:
# Parameters for analyses

# Gabor
omega_max_plot = 3.5 # [a.u.]
Tmin_Gabor = 20.     # [a.u.]
Tmax_Gabor = 380.    # [a.u.]


# Energetic distribution
E_min = -0.6    # [a.u.] - minimal energy in the analysis
E_max = 5.      # [a.u.] - maximal energy in the analysis
N_pts = 1500    # # of points in energy

# Numerical parameters
Nthreads = 10   # # of threads for the parallel computation

### Compute Gabor transform

The Gabor transform is one of the analyses provided directly by the C-library. 

In [ ]:
grad_V = output.get_sourceterm()
dt_Gabor = output.tgrid[1]-output.tgrid[0]
T = output.tgrid[output.Nt-1]

tgrid_Gabor, ogrid_Gabor, Gabor = DLL.gabor_transform(grad_V, dt_Gabor, output.Nt, omega_max_plot, Tmin_Gabor, Tmax_Gabor, 1000, a=8)

### Compute invariant energy distribution & ionisation probability (using `multiprocessing` parallelisation)

The computation of the invariant energy distribution is a computationally heavy task. It basically requires [to compute the photo-electron spectrum for each $t$](https://doi.org/10.1103/PhysRevA.106.053115https://doi.org/10.1103/PhysRevA.106.053115).

We use the `multiprocessing` module to compute the result in parallel. We need also some preparational computations: to find the ensemble of the bound states that will be projected out from the distribution. Then we define the computational routine to be parallelised by the `starmap_async`.

In [ ]:
# compute bound states that will be projected out
Energy_guess = [-0.5789, -0.2537, -0.1425, -0.0890, -0.0613, -0.0440, -0.0335, -0.0265, -0.0213, -0.0175, -0.0145, -0.0105, -0.0080, -0.0065, -0.0050, -0.0035]
inputs_array = []
GS = []
for i, E in enumerate(Energy_guess):
    inputs_array.append(inputs_def())
    inputs_array[i].init_default_inputs(Eguess=E, num_r=inputs.num_r, trg_a=trg_a, dt = 0.25, CV = 1e-15) # CV = 1e-15, else the resolvent does not converge for higher bound states
    DLL.init_GS(inputs_array[i]) 
    print("E_GS = {}".format(inputs_array[i].Einit))
    GS.append(inputs_array[i].get_GS())

GS = np.array(GS)

# Free the memory allocated by the temporary arrays for the energy computation
for i, E, in enumerate(inputs_array):
    inputs_array[i].delete(DLL)

In [ ]:
# define the function used for the parallelised computaiton of the photoelectron spectrum


t_psi, x_grid, wavefunction = output.get_wavefunction(inputs, grids=True)
wfs = wavefunction[0:-1:1]


def compute_PES_parallel(psi, GS, N_pts, E_min, E_max, jobID):
    for psi_b in GS:
        psi -= np.vdot(psi, psi_b)*psi_b # Remove the bound states using projection (note: np.vdot(a, b) == np.dot(np.conj(a), b) )

    ### Convert psi into a C-comprehensible array - flatten first, alternate real 
    # and imaginary parts, then return pointer to array
    #psi = np.array([psi.real, psi.imag]).transpose().flatten()
    #psi_ptr = ctypes_arr_ptr(c_double, len(psi), psi)

    ### Compute photoelectron spectrum for range [E_min, E_max]
    Energy = np.linspace(E_min, E_max, N_pts)
    Estep = Energy[1]-Energy[0]
    E_grid, PES = DLL.compute_PES(inputs, psi, num_E=len(Energy), Estep=Estep)

    # print("Job {} done.".format(jobID))
    print(f'Job {jobID} done.    \r', end="")
    
    ### Store the result
    return E_grid, PES

In [ ]:
# compute the analysis in parallel
from multiprocess import Pool
p = Pool(Nthreads)

map_ = [(wf, GS, N_pts, E_min, E_max, i+1) for i, wf in enumerate(wavefunction)]
result = p.starmap_async(compute_PES_parallel, map_)    # run calculation in parallel

PES = result.get()
PES_array = [PES[i][1] for i in range(len(PES))]        # collect the results

### Final plot of all data

In [ ]:
# code to generate the following figure
import matplotlib.colors as colors
fig = plt.figure(figsize=(14, 10))

# Define subplots using subplot2grid
ax1 = plt.subplot2grid((3, 2), (0, 0))  # Upper left
ax2 = plt.subplot2grid((3, 2), (0, 1))  # Upper right
ax3 = plt.subplot2grid((3, 2), (1, 0))  # Middle left
ax4 = plt.subplot2grid((3, 2), (1, 1))  # Middle right
ax5 = plt.subplot2grid((3, 2), (2, 0))  # Lower left
ax6 = plt.subplot2grid((3, 2), (2, 1))  # Lower right


ax1.plot(output.get_tgrid(),output.get_Efield(),label='Electric field')
ax1.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax1.set_ylabel(r'$\mathcal{E}~[\mathrm{a.u.}]$')
# ax1.set_title('Eelctric field')
ax1.legend()


ogrid = output.get_omegagrid()[:]
ko_max = mn.FindInterval(ogrid,omega_max_plot)
ax2.semilogy(mn.ConvertPhoton(ogrid[:ko_max],'omegaau','eV'), np.abs(output.get_Fsourceterm())[:ko_max],label='dipole acceleration spectrum')
ax2.set_xlim(mn.ConvertPhoton(ogrid[:ko_max],'omegaau','eV')[[0,-1]])
ax2.set_xlabel(r'$\omega~[\mathrm{eV}]$')
ax2.set_ylabel(r'$|(\partial \hat{\jmath}/\partial t)(\omega)|~[\mathrm{arb.~u.}]$')
ax2.legend()
# plt.xlim(0,3.5)


x_range = (np.abs(x_grid) < 250.1)
pc3 = ax3.pcolormesh(t_psi, x_grid[x_range], np.transpose(np.abs(wavefunction))[x_range],
                     cmap = 'jet',
                     norm = colors.LogNorm(vmin=1e-8, vmax=0.5),
                     shading = 'gouraud')
ax3.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax3.set_ylabel(r'$x~[\mathrm{a.u.}]$')
cbar2 = fig.colorbar(pc3, ax=ax3) #, orientation='horizontal')
cbar2.ax.set_ylabel(r'$|\psi|$ [a.u.]', rotation=90)



pc4 = ax4.pcolormesh(tgrid_Gabor, mn.ConvertPhoton(ogrid_Gabor,'omegaau','eV'), Gabor,
                     cmap = 'jet',
                     # vmin = 1e-6,
                     norm = colors.LogNorm(vmin=1e-6, vmax=1), # Normalize
                     shading = 'gouraud')
ax4.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax4.set_ylabel(r'Energy [eV]')
cbar4 = fig.colorbar(pc4, ax=ax4) #, orientation='horizontal')
cbar4.ax.set_ylabel(r'Gabor (spectrogram) [arb. u.]', rotation=90)


# plot_colormap(ogrid_Gabor, tgrid_Gabor, Gabor, z_min=1e-6, figsize=(5, 3))

pc5 = ax5.pcolormesh(t_psi,
                     mn.ConvertPhoton(np.linspace(E_min, E_max, N_pts),'omegaau','eV'),
                     1e4*np.array(PES_array).transpose(),
                     cmap = 'jet', #'bwr',
                     # vmin = 1e-6,
                     norm = colors.Normalize(vmin=1e4*1e-8, vmax=1e4*1e-4), # Normalize
                     shading = 'gouraud')

ax5.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax5.set_ylabel(r'Energy [eV]')
cbar5 = fig.colorbar(pc5, ax=ax5) #, orientation='horizontal')
cbar5.ax.set_ylabel("ivnariant electron's energy [arb. u.]", rotation=90)


ax6.plot(output.get_tgrid(),output.get_PopInt(),label=r'Volumetric GS population ($\int_{V_{\text{atom}}} |\psi(t)|^2$)')
ax6.plot(output.get_tgrid(),output.get_PopTot(),label=r'Invariant projected GS pop. ($|\langle\psi(t)|\psi_0 \rangle_{\text{inv}}|^2$)')
ax6.set_xlim(output.get_tgrid()[[0,-1]])
ax6.set_xlabel(r'$t~[\mathrm{a.u.}]$')
ax6.set_ylabel(r'Probability [-]')
ax6.legend()


fig.tight_layout()
plt.show()